In [ ]:
from pathlib import Path
from utils.get_input import get_input

# inp = Path("./test_input").read_text()
inp = get_input(5)
inp

'53|62\n43|48\n43|99\n62|76\n62|19\n62|85\n58|55\n58|84\n58|43\n58|64\n98|82\n98|47\n98|73\n98|49\n98|66\n99|97\n99|14\n99|58\n99|57\n99|92\n99|49\n73|84\n73|49\n73|57\n73|87\n73|39\n73|59\n73|35\n92|82\n92|39\n92|83\n92|93\n92|84\n92|31\n92|53\n92|46\n87|36\n87|35\n87|38\n87|82\n87|58\n87|83\n87|11\n87|57\n87|53\n86|91\n86|23\n86|19\n86|16\n86|71\n86|62\n86|47\n86|31\n86|76\n86|97\n83|89\n83|86\n83|93\n83|84\n83|77\n83|64\n83|43\n83|36\n83|98\n83|48\n83|45\n91|76\n91|71\n91|69\n91|49\n91|38\n91|16\n91|14\n91|31\n91|35\n91|47\n91|73\n91|59\n84|61\n84|93\n84|99\n84|23\n84|98\n84|62\n84|66\n84|19\n84|11\n84|86\n84|32\n84|48\n84|43\n47|46\n47|31\n47|27\n47|92\n47|59\n47|39\n47|57\n47|36\n47|35\n47|82\n47|64\n47|84\n47|87\n47|71\n34|53\n34|69\n34|36\n34|92\n34|43\n34|58\n34|49\n34|87\n34|31\n34|39\n34|83\n34|84\n34|82\n34|27\n34|47\n89|23\n89|48\n89|77\n89|91\n89|76\n89|98\n89|51\n89|86\n89|97\n89|61\n89|69\n89|73\n89|16\n89|99\n89|32\n89|47\n71|31\n71|32\n71|45\n71|93\n71|53\n71|57\n71|36

In [ ]:
# %%timeit
rules, updates = inp.split("\n\n")

updates = [[int(xx) for xx in x.split(",")] for x in updates.split("\n")]

rules = [[int(xx) for xx in x.split("|")] for x in rules.split("\n")]

rules_map_k_before_v = {}
for r1, r2 in rules:
    if r1 in rules_map_k_before_v:
        rules_map_k_before_v[r1].append(r2)
    else:
        rules_map_k_before_v[r1] = [r2]

rules_map_v_before_k = {}
for k, v in rules_map_k_before_v.items():
    for i in v:
        if i in rules_map_v_before_k:
            rules_map_v_before_k[i].append(k)
        else:
            rules_map_v_before_k[i] = [k]

In [47]:
def part1():
    sum = 0

    for update in updates:
        # print(update)
        past = set()
        whole_row = set(update)
        rules_followed = False

        for p in update:
            should_before_p = (
                rules_map_v_before_k[p] if p in rules_map_v_before_k else []
            )
            should_before_p = set(should_before_p).intersection(whole_row)
            rules_followed = set(should_before_p).issubset(past)
            if not rules_followed:
                # print("RULE MISSED", p, should_before_p, past)
                break
            # print(p, rules_followed)
            past.add(p)

        if rules_followed:
            # print("YES")
            sum += update[len(update) // 2]

    return sum


part1()

7307

In [48]:
%%timeit
part1()

1.73 ms ± 19.9 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [ ]:
def part2():
    sum = 0

    for update in updates:
        # print("\nUPDATE: ", update)
        past = set()
        whole_row = set(update)

        rules_followed_all = True
        i = 0
        while i < len(update):

            p = update[i]
            should_before_p = (
                rules_map_v_before_k[p] if p in rules_map_v_before_k else []
            )
            should_before_p = set(should_before_p).intersection(whole_row)
            rules_followed = set(should_before_p).issubset(past)
            past.add(p)
            if not rules_followed:
                rules_followed_all = False
                # print("RULE MISSED", p, should_before_p, past)
                move_before_p = should_before_p - past
                # print(f"Moving {move_before_p} before {p}")
                update = (
                    update[:i]
                    + list(move_before_p)
                    + [x for x in update[i:] if x not in move_before_p]
                )
                # print("NEW UPDATE", update)
                i = 0
                past = set()
            # print(p, rules_followed)

            i += 1

        if not rules_followed_all:
            to_add = update[len(update) // 2]
            # print("YES, adding", to_add)
            sum += to_add

    return sum


part2()

4713

In [51]:
%%timeit
part2()

5.73 ms ± 143 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)
